# Homework: Build Your Own NN Framework

The goal of this homework is simple, yet an actual implementation may take some time. We are going to write an Artificial Neural Network (almost) from scratch. The software design was heavily inspired by [PyTorch](http://pytorch.org) which is the main framework of our course.

Your homework task comprises 4 steps:
 - Implement all modules in `homework_part1_modules.ipynb` . These modules will be checked automatically using tests provided in `homework_part1_test_modules.ipynb`
 - Check that your modules work on 'toy example' (see below)
 - Train small neural network to classify images from mnist dataset
 - Train the same neural network in pytorch

In [ ]:
%matplotlib inline
from time import time, sleep
import numpy as np
import matplotlib.pyplot as plt
from IPython import display

# 1. Framework

Implement everything in `homework_part1_modules.ipynb`. Read all the comments thoughtfully, it will reduce the amount of time spended on debug. Please don't change the prototypes since layers you implement will be automatically checked.

The typical assumption is that `module.backward` is always executed after `module.forward`,
so layer output is stored in `_output`. This would be useful for `SoftMax`. 

### Tech note about inplace operations
You can use `np.multiply`, `np.add`, `np.divide`, `np.subtract` with parameter `out` filled instead of `*`,`+`,`/`,`-` for better memory handling through inplace operations

Example: suppose you allocated a variable 

```
a = np.zeros(...)
```
So, instead of
```
a = b + c  # will be reallocated, GC needed to free
``` 
You can use: 
```
np.add(b, c, out=a) # puts result in `a`
```

In [ ]:
# (re-)load layers
%run homework_part1_modules.ipynb

# 2. Toy example

Once you have implemented the modules, you can check that your framework can actually be used for neural network training. Use this example to debug your code, start with logistic regression and then test other layers. You do not need to change anything here. This code is provided for you to test the layers. Also it is easy to use this code in MNIST task.

## Dataset

In [ ]:
# Generate some data
n_samples = 500

X_pos = np.random.randn(n_samples, 2) + np.array([2, 2])
X_neg = np.random.randn(n_samples, 2) + np.array([-2, -2])
X = np.vstack([X_pos, X_neg])

Y = np.concatenate([np.ones(n_samples), np.zeros(n_samples)])[:, np.newaxis]
Y = np.hstack([Y, 1 - Y])  # one-hot encoding

plt.scatter(X[:, 0],X[:, 1], c=Y[:, 0], edgecolors='none')

## Neural network architecture

Define a **logistic regression** for debugging. 

In [ ]:
net = Sequential()
net.add_module(Linear(2, 2))
net.add_module(LogSoftMax())

criterion = ClassNLLCriterion()

print(net)

# Test something like that then 

#net = Sequential()
#net.add_module(Linear(2, 4))
#net.add_module(ReLU())
#net.add_module(Linear(4, 2))
#net.add_module(LogSoftMax())

## Optimizer

Let's start with simple SGD with momentum. You can test Adam if sgd works fine

In [ ]:
opt = SGD(net, lr=1e-1, momentum=0.9)
#opt = Adam(net)

## Training

Start with batch_size = 1000 to make sure every step lowers the loss, then try stochastic version.

In [ ]:
# batch generator
def get_batches(dataset, batch_size):
    X, Y = dataset
    n_samples = X.shape[0]
        
    # Shuffle at the start of epoch
    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    
    for start in range(0, n_samples, batch_size):
        end = min(start + batch_size, n_samples)
        
        batch_idx = indices[start:end]
    
        yield X[batch_idx], Y[batch_idx]

Basic training loop. Examine it.

In [ ]:
def train_network(net, optimizer, X, Y, batch_size, n_epoch):
    loss_history = []

    for i in range(n_epoch):
        for x_batch, y_batch in get_batches((X, Y), batch_size):

            net.zero_grad()

            # Forward
            predictions = net.forward(x_batch)
            loss = criterion.forward(predictions, y_batch)

            # Backward
            dp = criterion.backward(predictions, y_batch)
            net.backward(x_batch, dp)

            # Update weights
            optimizer.step()

            loss_history.append(loss)

        # Visualize
        display.clear_output(wait=True)
        plt.figure(figsize=(8, 6))

        plt.title("Training loss")
        plt.xlabel("#iteration")
        plt.ylabel("loss")
        plt.plot(loss_history, 'b')
        plt.show()

        print('Current loss: %f' % loss)    

In [ ]:
train_network(net, opt, X, Y, batch_size=128, n_epoch=20)

# 3. Classification on MNIST

We are using old good [MNIST](http://yann.lecun.com/exdb/mnist/) as our dataset.

In [ ]:
import mnist
X_train, y_train, X_val, y_val, X_test, y_test = mnist.load_dataset()

In [ ]:
n_samples = 5
plt.figure(figsize=(10,4))
for i in range(n_samples):
    plt.subplot(1,n_samples,i+1)
    plt.imshow(X_train[i], 'gray')
    plt.title('label:{}'.format(y_train[i]))

Your task:
1. Preprocess data: One-hot encode the labels and reshape 28x28 images to 1D vectors of size 784
2. Define simple network Linear(128) + relu + Linear(10) + LogSoftMax
3. Define criterion ClassNLLCriterion
4. Define optimizer (sgd or adam)
5. Train network
6. Test network on test data, print its accuracy. Make sure accuracy is greater than 95% on test set

In [ ]:
print(X_train.shape, y_train.shape, y_train[:10])

# YOUR CODE: reshape + one-hot encoding

X_test = ...
X_train = ...

y_train = ...
y_test = ...

assert len(X_test.shape) == 2, X_test.shape
assert len(X_test[-1]) == 28*28, X_test.shape
assert len(y_test.shape) == 2, y_test.shape
assert len(y_test[-1]) == 10, y_test.shape

assert len(X_train.shape) == 2, X_train.shape
assert len(X_train[-1]) == 28*28, X_train.shape
assert len(y_train.shape) == 2, y_train.shape
assert len(y_train[-1]) == 10, y_train.shape

In [ ]:
# YOUR CODE: network definition Linear(128) + relu + Linear(10) + LogSoftMax
net = Sequential()
net.add_module(...)
...

print(net)

In [ ]:
# YOUR CODE: criterion definition
criterion = ...

In [ ]:
# YOUR CODE: optimizer definition
# for sgd normally you can start with momentum = 0.9 or 0.99 and lr=1e-2  or 1e-1
opt = ...

In [ ]:
# YOUR CODE: train network, try batch 128 and 5-15 epochs
train_network(net, ...

Test function is already written here for you. Check your network accuracy, it should be above 95% on test set to pass the grade.

In [ ]:
def test_network(network, x_test, y_test, batch_size=1):
    n_correct = 0
    loss_hist = []
    for x_batch, y_batch in get_batches((x_test, y_test), batch_size):
        # Forward
        predictions = network.forward(x_batch)
        loss = criterion.forward(predictions, y_batch)
        loss_hist.append(loss)
        
        y_pred = np.argmax(predictions, axis=-1)
        y_true = np.argmax(y_batch, axis=-1)
        n_correct += np.sum(y_pred == y_true)
    print('Mean loss: {}'.format(np.mean(loss_hist)))    
    accuracy = n_correct * 100. / len(y_test)
    print('accuracy: {}'.format(accuracy))
    return accuracy

In [ ]:
accuracy = test_network(net, X_test, y_test)
assert accuracy > 95, accuracy

# 4. Pytorch on MNIST

Let's do the same network in pytorch

0. Preprocess data. This time gt labels should represent class indices and images should be reshaped from 28x28 images to 1D vectors of size 784.
1. Define network`Linear(784, 128) -> Relu -> Linear(128, 10) -> LogSoftMax`
  - you can find all layers in `torch.nn` module
  - the only difference from our framework will be in `Sequential.add_module` method, check its signature in documentation
2. Define criterion
  - analogue of `ClassNLLCriterion` is `torch.nn.NLLoss`, but it takes class labeles as the second parameter instead of one-hot encoded labels - that's why we need different preprocessing on step 0.
3. Define optimizer from torch.optim module
  - optimizers take a list of network parameters instead of the whole network as in our framework. You can easily get the parameters by calling `net.parameters()` method
4. Rewrite `train_network()` and `test_network()` functions so that they will be able work with pytorch network
  - just convert numpy arrays with batch data to `torch.Tensor` using `torch.from_numpy` and feed them to the network
  - result tensors can easily converted from pytorch to numpy using chain of two methods: `detach().numpy()`
  - 
5. Train your network
6. Test network on test data, print its accuracy. Make sure accuracy is greater than 95% on test set

In [ ]:
import mnist
X_train, y_train, X_val, y_val, X_test, y_test = mnist.load_dataset()

In [ ]:
# YOUR CODE: reshape X_train, X_test and cast y_train, y_test to int64
print(X_train.shape, y_train.shape, y_train[:10])

X_test = ...
X_train = ...

y_train = ...
y_test = ...

assert len(X_test.shape) == 2, X_test.shape
assert len(X_test[-1]) == 28*28, X_test.shape
assert len(y_test.shape) == 1, y_test.shape

assert len(X_train.shape) == 2, X_train.shape
assert len(X_train[-1]) == 28*28, X_train.shape
assert len(y_train.shape) == 1, y_train.shape

In [ ]:
import torch

In [ ]:
# YOUR CODE define network Linear(128) + relu + Linear(10) + LogSoftmax 
net = torch.nn.Sequential()
net.add_module('fc1', torch.nn.Linear(28*28, 128))
net.add_module(...)
...

print(net)

In [ ]:
# YOUR CODE: define criterion


In [ ]:
# YOUR CODE: define optimizer


In [ ]:
def train_network_pytorch(net, opt, X, Y, batch_size, n_epoch):
    loss_history = []

    for i in range(n_epoch):
        for x_batch, y_batch in get_batches((X, Y), batch_size):
            
            # YOUR CODE: rewrite train_network() function to be able to work with pytorch network
            pass

        # Visualize
        display.clear_output(wait=True)
        plt.figure(figsize=(8, 6))

        plt.title("Training loss")
        plt.xlabel("#iteration")
        plt.ylabel("loss")
        plt.plot(loss_history, 'b')
        plt.show()

        print('Current loss: %f' % loss)    
        
train_network_pytorch(net, opt, X_train, y_train, 128, 10)

In [ ]:
def test_network_pytorch(network, x_test, y_test, batch_size=1):
    n_correct = 0
    loss_hist = []

    for x_batch, y_batch in get_batches((x_test, y_test), batch_size):
        # YOUR CODE: rewrite test_network() function to be able to work with pytorch network
        pass

    print('Mean loss: {}'.format(np.mean(loss_hist)))    
    accuracy = n_correct * 100. / len(y_test)
    print('accuracy: {}'.format(accuracy))
    return accuracy

Check your network accuracy, it should be above 95% on test set to pass the grade.

In [ ]:
accuracy = test_network_pytorch(net, X_test, y_test)
assert accuracy > 95, accuracy